In [2]:
import pandas as pd
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\real_estate_dataset.csv")

In [3]:
df.head()


,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
0,1,143.635030,1,3,3,1967,1,1,48,8.297631,5.935734,602134.816747
1,2,287.678577,1,2,1,1949,0,1,37,6.061466,10.827392,591425.135386
2,3,232.998485,1,3,2,1923,1,0,14,2.911442,6.904599,464478.696880
3,4,199.664621,5,2,2,1918,0,0,17,2.070949,8.284019,583105.655996
4,5,89.004660,4,3,3,1999,1,0,34,1.523278,14.648277,619879.142523


In [4]:
df[df.isna().any(axis=1)]

,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price


In [5]:
traincol=df.columns
traincol=traincol.delete(loc=0)
traincol=traincol.delete(loc=-1)
X=df[traincol]
Y=df['Price']

In [6]:
import numpy as np
xarray=np.array(traincol)
yarray=np.array(traincol)

In [7]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(xarray,yarray,test_size=0.3,random_state=69)
Xtest,Xval,ytest,yval=train_test_split(Xtest,ytest,test_size=0.5,random_state=69)


In [8]:
from torch.utils.data import DataLoader,Dataset

class dataset(Dataset):
    def __init__(self,X,Y):
        self.X=X
        self.Y=X
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]
    
traindataset=dataset(X=Xtrain,Y=ytrain)
testdataset=dataset(X=Xtest,Y=ytest)
valdataset=dataset(X=Xval,Y=yval)



In [9]:
traindataloader=DataLoader(traindataset,batch_size=10,shuffle=True)
testdataloader=DataLoader(testdataset,batch_size=10,shuffle=True)
valdataloader=DataLoader(valdataset,batch_size=10,shuffle=True)

10

In [15]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model,self).__init__(*args, **kwargs)
        self.linear=nn.Linear(X.shape[1],1)
    
    def forward(self,x):
        x=self.linear(x)
        return x


model=Model()


In [23]:
from torchsummary import summary

summary(model,input_size=(10,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]              11
Total params: 11
Trainable params: 11
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [24]:
import torch.optim as optim
critetion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3)

In [ ]:
total_loss_train_plot=[]
total_loss_val_plot=[]
total_acc_train_plot=[]
total_acc_val_plot=[]

